In [3]:
import psycopg2
import json
from tqdm import tqdm

conn = psycopg2.connect(
    dbname   = "postgres",
    user     = "postgres",
    password = "password",
    host     = "localhost",
    port     = 5432
)

cur = conn.cursor()
cur.execute("""
    SELECT *
    FROM ag_catalog.cypher('gnd', $$
      MATCH (n)
      RETURN id(n) AS node_id, labels(n) AS labels, properties(n) AS properties
    $$) AS result(node_id bigint, labels ag_catalog.agtype, properties ag_catalog.agtype);
""")

nodes_data = cur.fetchall()
cur.close()
cur = conn.cursor()

cur.execute("""
    SET search_path TO ag_catalog;
    SELECT *
    FROM cypher('gnd', $$
      MATCH (start)-[r]->(target)
      RETURN id(start) AS source, id(target) as target, id(r) as rel
    $$) AS result(source bigint, target bigint, rel bigint);
""")
edges_data = cur.fetchall()

cur.close()
conn.close()

In [4]:
from pyvis.network import Network

net = Network(notebook = True, height = "750px", width="100%")

for node in tqdm(nodes_data, desc="Adding nodes"):
    node_id = node[0]
    labels = json.loads(node[1])
    properties = json.loads(node[2])

    label = properties.get('name', ', '.join(labels)) if properties else ', '.join(labels)

    net.add_node(node_id, label=label, titles=str(properties))

for edge in tqdm(edges_data, desc="Adding edges"):
    source = edge[0]
    target = edge[1]
    net.add_edge(source, target)

net.show("graph.html")

Adding nodes:   5%|▍         | 30454/618162 [00:05<01:40, 5851.64it/s] 


KeyboardInterrupt: 